# 최초 데이터 준비

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:

# Mip-NeRF 360 데이터셋 다운로드 (약 4~5GB)
!wget http://storage.googleapis.com/gresearch/refraw360/360_v2.zip -P /content/drive/MyDrive/3dgs


--2025-11-24 10:11:23--  http://storage.googleapis.com/gresearch/refraw360/360_v2.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.174.207, 74.125.23.207, 74.125.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.174.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12535427936 (12G) [application/zip]
Saving to: ‘/content/drive/MyDrive/3dgs/360_v2.zip’

360_v2.zip          100%[===================>]  11.67G  30.0MB/s    in 7m 8s   

2025-11-24 10:18:32 (27.9 MB/s) - ‘/content/drive/MyDrive/3dgs/360_v2.zip’ saved [12535427936/12535427936]



# 클론 및 의존성 설치

In [ ]:
# 1. 구글 드라이브 마운트 (데이터 및 결과 저장용)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

# 3. 깃허브 레포 클론 (사용자님 레포)
# 이미 클론되어 있다면 pull만 하거나 넘어갑니다.
if not os.path.exists("gaussian-splatting"):
    !git clone --recursive https://github.com/YuDongNam/gaussian-splatting.git

%cd gaussian-splatting


In [ ]:

# 4. 의존성 설치 및 서브모듈 컴파일 (약 3~5분 소요)
# 주의: 코랩 GPU에 맞춰 컴파일하므로 시간이 좀 걸립니다.
!pip install -q plyfile
!pip install -r requirements.txt
!pip install submodules/diff-gaussian-rasterization
!pip install submodules/simple-knn

# 데이터 준비(from gdrive to local)

In [ ]:
# Colab에서 예시
from src.config import CFG

cfg = CFG()  # 이 시점에 모든 폴더가 자동 생성됨!
# data/, data/ckpts/, data/tables/, outputs/ 모두 생성됨

## 1. Mip-NeRF 360 (6개 장면: bonsai, counter, bicycle, garden, stump, kitchen)

In [ ]:
import gdown

# 1. 링크에서 'd/' 와 '/view' 사이에 있는 파일 ID만 추출
file_id = "1lNQKolRBe3bHV2mz7uKcWtG0rB8vGABI"

# 2. 다운로드용 URL 형식으로 변환
url = f"https://drive.google.com/uc?id={file_id}"

# 3. 다운로드 실행 (output은 저장할 파일명, 지정하지 않으면 원본 이름 사용)
gdown.download(url, output="360v2.zip", quiet=False)

In [ ]:
# data/scenes 폴더 생성 (없으면 자동 생성)
import os
os.makedirs("data/scenes", exist_ok=True)

# 압축 해제 (scenes 폴더 안에)
!unzip 360v2.zip -d data/scenes

## 2. truck and temples (2개 장면: train, truck)

In [ ]:
import gdown

# 1. 링크에서 'd/' 와 '/view' 사이에 있는 파일 ID만 추출
file_id = "1MBn36eONnqHCPj0aeM34zh4t74TM2ZXw"

# 2. 다운로드용 URL 형식으로 변환
url = f"https://drive.google.com/uc?id={file_id}"

# 3. 다운로드 실행 (output은 저장할 파일명, 지정하지 않으면 원본 이름 사용)
gdown.download(url, output="traintruck.zip", quiet=False)

In [ ]:
# data/scenes 폴더가 없으면 생성
import os
os.makedirs("data/scenes", exist_ok=True)

# 압축 해제 (scenes 폴더 안에)
!unzip traintruck.zip -d data/scenes

# 장면 학습

In [ ]:
import subprocess
import os
import sys
from pathlib import Path

# ==========================================
# [설정 영역] 여기만 수정하면 됩니다.
# ==========================================

# 1. 데이터가 있는 상위 폴더 (scenes 폴더)
DATA_ROOT = "/content/gaussian-splatting/data/scenes"

# 2. 결과물을 저장할 폴더
OUTPUT_ROOT = "/content/gaussian-splatting/data/ckpts"

# 3. 학습할 장면 리스트 (8개)
SCENES = [
    "bonsai", "counter", "bicycle", "train",
    "garden", "stump", "kitchen", "truck"
]

# 4. 저장할 체크포인트 Iteration (5개 지점)
# 초기(7k), 중기(15k), 후기(20k, 25k), 수렴(30k)
SAVE_ITERS = [7000, 15000, 20000, 25000, 30000]

# 5. GPU 해상도 옵션 (메모리 부족 시 2 또는 4로 변경)
RESOLUTION_SCALE = 4

# 6. 최종 iteration (학습 완료 기준)
FINAL_ITER = 30000

# ==========================================

# 저장 옵션을 문자열로 변환
save_iter_str = " ".join(map(str, SAVE_ITERS))

def is_training_complete(scene_name: str, model_path: str) -> bool:
    """Check if training is already complete for a scene."""
    final_ckpt = Path(model_path) / "point_cloud" / f"iteration_{FINAL_ITER}" / "point_cloud.ply"
    return final_ckpt.exists()


In [ ]:

print(f"🚀 총 {len(SCENES)}개 장면에 대한 순차 학습을 시작합니다...\n")

for idx, scene in enumerate(SCENES, 1):
    source_path = os.path.join(DATA_ROOT, scene)
    model_path = os.path.join(OUTPUT_ROOT, scene)
    
    print(f"\n{'='*60}")
    print(f"[{idx}/{len(SCENES)}] 장면: {scene}")
    print(f"{'='*60}")
    print(f"  데이터 경로: {source_path}")
    print(f"  저장 경로: {model_path}")
    
    # 학습 완료 여부 확인
    if is_training_complete(scene, model_path):
        print(f"  ⏭️  {scene}는 이미 학습이 완료되어 건너뜁니다.")
        continue
    
    # 소스 경로 존재 확인
    if not os.path.exists(source_path):
        print(f"  ❌ 경고: {source_path}가 존재하지 않습니다. 건너뜁니다.")
        continue
    
    # 명령어 구성
    cmd = [
        "python", "train.py",
        "-s", source_path,
        "-m", model_path,
        "-r", str(RESOLUTION_SCALE),
        "--save_iterations"
    ] + list(map(str, SAVE_ITERS))
    
    # subprocess로 실행 (에러 처리 포함)
    try:
        print(f"  🏃 학습 시작...")
        result = subprocess.run(
            cmd,
            check=True,
            capture_output=False,  # 실시간 로그 출력
            text=True
        )
        print(f"  ✅ {scene} 학습 완료!")
        
    except subprocess.CalledProcessError as e:
        print(f"  ❌ {scene} 학습 중 오류 발생: {e}")
        print(f"  ⚠️  다음 장면으로 진행합니다...")
        continue
    except Exception as e:
        print(f"  ❌ 예상치 못한 오류: {e}")
        print(f"  ⚠️  다음 장면으로 진행합니다...")
        continue

print(f"\n{'='*60}")
print("🎉 모든 장면의 학습이 종료되었습니다.")
print(f"{'='*60}")

# 통계적 특징 추출 (Feature Extraction)


In [ ]:
import torch
import pandas as pd
import numpy as np
import json
import gc
from pathlib import Path
from tqdm import tqdm
import sys

# 프로젝트 경로 추가
sys.path.insert(0, '/content/gaussian-splatting')

from src.gs_adapter import load_gaussians, Gaussians
from src.features import eigen_stats, estimate_coverage_overlap
from src.run_experiments import load_cameras_from_json, build_view_matrix, build_projection_matrix

# GPU 메모리 정리
torch.cuda.empty_cache()
gc.collect()

print("✅ 통계 분석 모듈 로드 완료")


In [ ]:
def extract_features_from_checkpoint(
    scene_id: str,
    ckpt_path: Path,
    cameras_path: Path,
    iteration: int
) -> dict:
    """Extract statistical features from a single checkpoint."""
    
    results = {
        'scene_id': scene_id,
        'iteration': iteration,
        'N_g': 0,
        'coverage': 0.0,
        'mean_alpha': 0.0,
        'energy_density': 0.0,
        'anisotropy_median': 0.0,
        'anisotropy_mean': 0.0,
        'anisotropy_std': 0.0,
        'volume_median': 0.0,
        'psnr': None,  # Will be computed separately if possible
    }
    
    try:
        # Load Gaussians
        gs = load_gaussians(ckpt_path, max_sh_degree=3)
        N_g = gs.xyz.shape[0]
        results['N_g'] = N_g
        
        # Compute geometric statistics
        stats = eigen_stats(gs.scaling, gs.rotation)
        anisotropy = stats['anisotropy'].cpu().numpy()
        volume = stats['volume'].cpu().numpy()
        
        results['anisotropy_median'] = float(np.median(anisotropy))
        results['anisotropy_mean'] = float(np.mean(anisotropy))
        results['anisotropy_std'] = float(np.std(anisotropy))
        results['volume_median'] = float(np.median(volume))
        
        # Load cameras and compute coverage/overlap for first camera (representative)
        try:
            cameras = load_cameras_from_json(cameras_path)
            if len(cameras) > 0:
                cam = cameras[0]
                viewmat = build_view_matrix(
                    cam.get('position', [0, 0, 0]),
                    cam.get('rotation', [[1,0,0], [0,1,0], [0,0,1]])
                )
                projmat = build_projection_matrix(
                    cam.get('fx', 400.0),
                    cam.get('fy', 400.0),
                    cam.get('width', 800),
                    cam.get('height', 800)
                )
                
                metrics = estimate_coverage_overlap(
                    gs=gs,
                    viewmat=viewmat,
                    projmat=projmat,
                    image_height=cam.get('height', 800),
                    image_width=cam.get('width', 800),
                    alpha_threshold=0.01,
                    sh_degree=3
                )
                
                results['coverage'] = metrics['coverage']
                results['mean_alpha'] = metrics['mean_alpha']
                results['energy_density'] = metrics['energy_density']
        except Exception as e:
            print(f"    ⚠️  Coverage 계산 실패: {e}")
        
        # Clean up
        del gs, stats
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"    ❌ 특징 추출 실패: {e}")
        return None
    
    return results

print("✅ 특징 추출 함수 정의 완료")


In [ ]:
# 모든 장면과 체크포인트에 대해 통계 추출
all_results = []

print(f"\n{'='*60}")
print("📊 통계적 특징 추출 시작")
print(f"{'='*60}\n")

for scene in SCENES:
    scene_model_path = Path(OUTPUT_ROOT) / scene
    
    # cameras.json 경로 확인
    cameras_path = scene_model_path / "cameras.json"
    if not cameras_path.exists():
        print(f"⚠️  {scene}: cameras.json을 찾을 수 없습니다. 건너뜁니다.")
        continue
    
    print(f"\n📁 장면: {scene}")
    
    # 각 체크포인트 iteration에 대해 처리
    for iteration in SAVE_ITERS:
        ckpt_path = scene_model_path / "point_cloud" / f"iteration_{iteration}" / "point_cloud.ply"
        
        if not ckpt_path.exists():
            print(f"  ⏭️  iteration_{iteration}: 체크포인트 없음")
            continue
        
        print(f"  🔍 iteration_{iteration} 분석 중...")
        
        try:
            result = extract_features_from_checkpoint(
                scene_id=scene,
                ckpt_path=ckpt_path,
                cameras_path=cameras_path,
                iteration=iteration
            )
            
            if result is not None:
                all_results.append(result)
                print(f"    ✅ 완료: N_g={result['N_g']}, Coverage={result['coverage']:.4f}")
            else:
                print(f"    ❌ 실패")
                
        except Exception as e:
            print(f"    ❌ 오류: {e}")
            continue
        
        # 메모리 정리
        torch.cuda.empty_cache()
        gc.collect()
    
    # 장면 처리 완료 후 메모리 정리
    torch.cuda.empty_cache()
    gc.collect()
    print(f"  ✅ {scene} 처리 완료\n")

print(f"\n{'='*60}")
print(f"📊 총 {len(all_results)}개 체크포인트 분석 완료")
print(f"{'='*60}")


In [ ]:
# DataFrame으로 변환 및 저장
if len(all_results) > 0:
    df = pd.DataFrame(all_results)
    
    # 컬럼 순서 정리
    column_order = [
        'scene_id', 'iteration', 'N_g',
        'coverage', 'mean_alpha', 'energy_density',
        'anisotropy_median', 'anisotropy_mean', 'anisotropy_std',
        'volume_median', 'psnr'
    ]
    df = df[[col for col in column_order if col in df.columns]]
    
    # 결과 저장
    output_csv = Path(OUTPUT_ROOT) / "records.csv"
    df.to_csv(output_csv, index=False)
    
    print(f"\n💾 결과 저장 완료: {output_csv}")
    print(f"\n📈 요약 통계:")
    print(df.groupby('scene_id').agg({
        'N_g': 'max',
        'coverage': ['mean', 'max'],
        'anisotropy_median': 'mean'
    }))
    
    print(f"\n📊 전체 데이터프레임:")
    print(df)
else:
    print("⚠️  추출된 결과가 없습니다.")


## (선택사항) PSNR 계산

render.py를 사용하여 test set에 대한 PSNR을 계산할 수 있습니다.


In [ ]:
# PSNR 계산 (선택사항 - 시간이 오래 걸릴 수 있음)
# 각 장면의 최종 iteration(30000)에 대해서만 PSNR 계산

COMPUTE_PSNR = False  # True로 설정하면 PSNR 계산 수행

if COMPUTE_PSNR:
    print(f"\n{'='*60}")
    print("📸 PSNR 계산 시작 (시간이 오래 걸릴 수 있습니다)")
    print(f"{'='*60}\n")
    
    psnr_results = {}
    
    for scene in SCENES:
        scene_model_path = Path(OUTPUT_ROOT) / scene
        source_path = Path(DATA_ROOT) / scene
        
        if not scene_model_path.exists() or not source_path.exists():
            continue
        
        print(f"📸 {scene} 렌더링 중...")
        
        try:
            # render.py 실행 (test set만 렌더링)
            cmd = [
                "python", "render.py",
                "-s", str(source_path),
                "-m", str(scene_model_path),
                "--iteration", "30000",
                "--skip_train",  # train set 건너뛰기
                "--quiet"
            ]
            
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                timeout=3600  # 1시간 타임아웃
            )
            
            # metrics.py로 PSNR 계산
            metrics_cmd = [
                "python", "metrics.py",
                "-m", str(scene_model_path)
            ]
            
            metrics_result = subprocess.run(
                metrics_cmd,
                capture_output=True,
                text=True,
                timeout=300
            )
            
            # PSNR 파싱 (간단한 방법)
            # 실제로는 metrics.py의 출력을 파싱해야 함
            print(f"  ✅ {scene} 렌더링 완료")
            
        except subprocess.TimeoutExpired:
            print(f"  ⏱️  {scene} 타임아웃")
        except Exception as e:
            print(f"  ❌ {scene} 오류: {e}")
        
        torch.cuda.empty_cache()
        gc.collect()
    
    print("\n✅ PSNR 계산 완료")
else:
    print("\n⏭️  PSNR 계산 건너뜀 (COMPUTE_PSNR=False)")


In [ ]:
# results.json NaN 문제 진단 및 해결

import json
import os
from pathlib import Path

print(f"\n{'='*60}")
print("🔍 results.json NaN 문제 진단")
print(f"{'='*60}\n")

for scene in SCENES:
    scene_model_path = Path(OUTPUT_ROOT) / scene
    results_json_path = scene_model_path / "results.json"
    
    if not results_json_path.exists():
        print(f"⚠️  {scene}: results.json이 없습니다.")
        continue
    
    print(f"\n📁 {scene} 진단 중...")
    
    # results.json 읽기
    with open(results_json_path, 'r') as f:
        results_data = json.load(f)
    
    scene_key = str(scene_model_path)
    if scene_key not in results_data:
        print(f"  ⚠️  results.json에 {scene_key}가 없습니다.")
        continue
    
    # 각 iteration 확인
    for iteration in SAVE_ITERS:
        method_key = f"ours_{iteration}"
        
        if method_key not in results_data[scene_key]:
            print(f"  ⚠️  {scene} iteration_{iteration}: method '{method_key}'가 없습니다.")
            continue
        
        psnr_value = results_data[scene_key][method_key].get("PSNR")
        
        if psnr_value is None or (isinstance(psnr_value, float) and (psnr_value != psnr_value)):  # NaN check
            print(f"\n  🔍 {scene} iteration_{iteration} NaN 원인 확인:")
            
            # 렌더링 결과 폴더 확인
            renders_dir = scene_model_path / "test" / method_key / "renders"
            gt_dir = scene_model_path / "test" / method_key / "gt"
            
            print(f"    📂 렌더링 폴더: {renders_dir}")
            print(f"    📂 GT 폴더: {gt_dir}")
            
            renders_exist = renders_dir.exists()
            gt_exist = gt_dir.exists()
            
            if not renders_exist:
                print(f"    ❌ 렌더링 폴더가 없습니다!")
                print(f"       해결: render.py를 다시 실행하세요.")
                continue
            
            if not gt_exist:
                print(f"    ❌ GT 폴더가 없습니다!")
                print(f"       해결: render.py를 다시 실행하세요.")
                continue
            
            # 이미지 파일 확인
            render_files = list(renders_dir.glob("*.png")) if renders_dir.exists() else []
            gt_files = list(gt_dir.glob("*.png")) if gt_dir.exists() else []
            
            print(f"    📊 렌더링 이미지 개수: {len(render_files)}")
            print(f"    📊 GT 이미지 개수: {len(gt_files)}")
            
            if len(render_files) == 0:
                print(f"    ❌ 렌더링 이미지가 없습니다!")
                print(f"       해결: render.py를 다시 실행하세요.")
                continue
            
            if len(gt_files) == 0:
                print(f"    ❌ GT 이미지가 없습니다!")
                print(f"       해결: render.py를 다시 실행하세요.")
                continue
            
            if len(render_files) != len(gt_files):
                print(f"    ⚠️  이미지 개수가 일치하지 않습니다!")
                print(f"       렌더링: {len(render_files)}개, GT: {len(gt_files)}개")
                print(f"       해결: render.py를 다시 실행하세요.")
                continue
            
            # 파일명 매칭 확인
            render_names = {f.stem for f in render_files}
            gt_names = {f.stem for f in gt_files}
            
            if render_names != gt_names:
                print(f"    ⚠️  파일명이 일치하지 않습니다!")
                print(f"       렌더링에만 있는 파일: {render_names - gt_names}")
                print(f"       GT에만 있는 파일: {gt_names - render_names}")
                print(f"       해결: render.py를 다시 실행하세요.")
                continue
            
            print(f"    ✅ 폴더와 이미지는 정상입니다.")
            print(f"    ⚠️  metrics.py 계산에 문제가 있을 수 있습니다.")
            print(f"       해결: metrics.py를 다시 실행하세요.")
            
            # 수동으로 metrics.py 재실행 제안
            print(f"\n    💡 수동 재실행:")
            print(f"       python metrics.py -m {scene_model_path}")

print(f"\n{'='*60}")
print("✅ 진단 완료")
print(f"{'='*60}\n")